In [73]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score
import matplotlib.pyplot as plt

In [74]:
housing = pd.read_csv('../Datasets/Housing.csv')
housing.head()

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [75]:
le = LabelEncoder()
housing['driveway'] = le.fit_transform(housing['driveway'])
housing['recroom'] = le.fit_transform(housing['recroom'])
housing['fullbase'] = le.fit_transform(housing['fullbase'])
housing['gashw'] = le.fit_transform(housing['gashw'])
housing['airco'] = le.fit_transform(housing['airco'])
housing['prefarea'] = le.fit_transform(housing['prefarea'])
housing

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,1,0,1,0,0,1,0
1,38500.0,4000,2,1,1,1,0,0,0,0,0,0
2,49500.0,3060,3,1,1,1,0,0,0,0,0,0
3,60500.0,6650,3,1,2,1,1,0,0,0,0,0
4,61000.0,6360,2,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
541,91500.0,4800,3,2,4,1,1,0,0,1,0,0
542,94000.0,6000,3,2,4,1,0,0,0,1,0,0
543,103000.0,6000,3,2,4,1,1,0,0,1,1,0
544,105000.0,6000,3,2,2,1,1,0,0,1,1,0


UNSCALED

In [76]:
# X = housing.drop('price', axis=1)
# y = housing['price']

# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=25)

# scores = []
# for i in range(1,11):
#     knn = KNeighborsRegressor(n_neighbors=i)
#     knn.fit(X_train,y_train)
#     y_pred = knn.predict(X_test)
#     scores.append([i,r2_score(y_test, y_pred)])

# scores = pd.DataFrame(scores, columns=['Neighbours', 'R2 score'])
# scores.sort_values('R2 score', ascending=False)

STANDARD SCALING

In [77]:
X = housing.drop('price', axis=1)
y = housing['price']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=25)


std = StandardScaler()
X_trn_scl = std.fit_transform(X_train)
X_tst_scl = std.transform(X_test)
print(len(X_trn_scl[0]))

scores = []
for i in range(1,11):
    knn = KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_trn_scl,y_train)
    y_pred = knn.predict(X_tst_scl)
    scores.append([i,r2_score(y_test, y_pred)])

scores = pd.DataFrame(scores, columns=['Neighbours', 'R2 score'])
scores.sort_values('R2 score', ascending=False)

11


,Neighbours,R2 score
3,4,0.576697
4,5,0.572285
9,10,0.566364
5,6,0.560651
8,9,0.554606
7,8,0.553857
6,7,0.543244
2,3,0.490856
1,2,0.373122
0,1,0.186813


In [78]:
test = pd.read_csv('../Datasets/tst_Housing.csv')
test['driveway'] = le.fit_transform(test['driveway'])
test['recroom'] = le.fit_transform(test['recroom'])
test['fullbase'] = le.fit_transform(test['fullbase'])
test['gashw'] = le.fit_transform(test['gashw'])
test['airco'] = le.fit_transform(test['airco'])
test['prefarea'] = le.fit_transform(test['prefarea'])


# *INFRENCING*

In [79]:
X_test_dum = std.transform(test)
X_test_dum
knn = KNeighborsRegressor(n_neighbors=4)
knn.fit(X_trn_scl,y_train)
y_pred = knn.predict(X_test_dum)
y_pred

array([48500.  , 47750.  , 36786.25, 64250.  ])

# BUT ABOVE METHOD IS NOT WHAT SIR USED, SIR's METHOD BELOW:

In [80]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

housing = pd.read_csv('../Datasets/Housing.csv')
housing.head()

X = housing.drop('price', axis=1)
categ_cols = list(X.columns[X.dtypes == object])
num_cols = list(X.columns[X.dtypes != object])

ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
trns = make_column_transformer((ohe, categ_cols), remainder='passthrough', verbose_feature_names_out=False)
trns = trns.set_output(transform='pandas')
X_trns = trns.fit_transform(X)

y = housing['price']
X_train, X_test, y_train, y_test = train_test_split(X_trns,y,test_size=0.3,random_state=25)

std = StandardScaler()
X_trn_scl = std.fit_transform(X_train)
X_tst_scl = std.transform(X_test)


knn = KNeighborsRegressor(n_neighbors=4)
knn.fit(X_trn_scl,y_train)
y_pred = knn.predict(X_tst_scl)

print(r2_score(y_test,y_pred))


0.5766971722542311


# SIR's METHOD INFRENCING

In [81]:

test = pd.read_csv("../Datasets/tst_Housing.csv")
X_test_dum = trns.transform(test)
X_test_dum = std.transform(X_test_dum)


y_pred = knn.predict(X_test_dum)
for i in y_pred:
    print(i)

58625.0
48475.0
46125.0
74250.0


MIN MAX SCALING

In [82]:
# X = housing.drop('price', axis=1)
# y = housing['price']
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=25)


# mm = MinMaxScaler()
# X_trn_scl = std.fit_transform(X_train)
# X_tst_scl = std.transform(X_test)


# scores = []
# for i in range(1,11):
#     knn = KNeighborsRegressor(n_neighbors=i)
#     knn.fit(X_trn_scl,y_train)
#     y_pred = knn.predict(X_tst_scl)
#     scores.append([i,r2_score(y_test, y_pred)])

# scores = pd.DataFrame(scores, columns=['Neighbours', 'R2 score'])
# scores.sort_values('R2 score', ascending=False)